In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import math, re, os
import tensorflow as tf
import numpy as np
import json
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    strategy = tf.distribute.MirroredStrategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.32.91.130:8470
INFO:tensorflow:Initializing the TPU system: grpc://10.32.91.130:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.32.91.130:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [ ]:
GCS_PATH = 'gs://dldaun/landmark' ## Google cloud bucket 주소

# validation 추가
TRAINING_FILENAMES = tf.io.gfile.glob(os.path.join(GCS_PATH, 'test_with_valid/train*'))
VALIDATION_FILENAMES = tf.io.gfile.glob(os.path.join(GCS_PATH, 'test_with_valid/val*'))
# TRAINING_FILENAMES = tf.io.gfile.glob(os.path.join(GCS_PATH, 'train/train*'))
# VALIDATION_FILENAMES = tf.io.gfile.glob(os.path.join(GCS_PATH, 'train/val*'))
TEST_FILENAMES = tf.io.gfile.glob(os.path.join(GCS_PATH, 'test/test*'))

# PRETRAINED_PATH = None  ## from scratch
# PRETRAINED_PATH = os.path.join(GCS_PATH, 'pretrained/eff7') ## 이전 학습 weights 위치
OUTPUT_PATH = os.path.join(GCS_PATH, 'output/dj1')
# OUTPUT_PATH = os.path.join('data/output/')

CLASSES = ['금천구청', '두리랜드', '이대원 장군묘', '성공회대학교', '송상현공 동상', '진위향교', '추계예술대학교', '계명대학교 행소박물관', '태조 이성계상', '마포 농수산물 시장', 
           '대구서구문화회관', '쉐라톤 서울 디큐브 시티 호텔', '구리시 교문도서관', '의정부시 상설 야외무대', '연지어린이공원', '종로성당', '대화 레포츠공원 체육시설', '하나로장로교회', '서울대학교 관악캠퍼스', '서울광장', 
           '영진종합사회복지관', '한국항공대학교 우주박물관', '좌동재래시장', 'NC백화점 안산고잔점', '청계천발원지', '김포 독립운동 기념관', '명동성당', '경춘선숲길', '유니온타워', '황희선생 유적지', 
           '신포국제시장', '안중근의사 기념관', '의정부 정보도서관', '인천 계양도서관', '와우정사', '덕소 도서관', '은평역사한옥박물관', '한중원 쉼터', '무등경기장 야구장', '한국지역난방공사 경기지사', 
           '절두산 순교 성지', '안양문화 예술재단', '문묘', '뚝섬한강공원', '관음정사', '농업 박물관', '육탄 이희복용사 동상', '망원시장', '의정부 성당', '연수어린이도서관', 
           '하남 시립 나룰도서관', '은평 한옥마을', '병암서원', '영동대교', '스타필드 고양', '신촌세브란스병원', '권율장군 대첩비', '구로구청', '백운포체육공원', '서오릉', 
           '성현교회', '배재학당역사박물관', '경희대학교 서울캠퍼스', '청계천', '한국기독교 100주년 기념탑', '장위전통시장', '부산시립미술관', '중구생활사전시관', '대학공원', '용인 세브란스 병원', 
           '타임스퀘어', '최사립 효자정각', '강남스타일 말춤 손목 조형물', '기건선생묘 및 신도비', '방화대교', '송포의 백송', '국제도덕협회부산지부', '함춘원', '롯데시네마[성서]', '신세계 백화점 의정부점', 
           '관훈갤러리', '낙성대공원', '롯데마트 연수점', 'DGB대구은행파크', '의정부시 청소년 수련관', '대한민국 역사 박물관', '웨스턴돔', '대구수목원', '시립 행신 어린이도서관', '군포시청', 
           '담다헌 체험교육관', '행복체육공원', '부평전투승전기념비', '수정4동 행정복지센터', '두산아트센터', '북촌 문화 센터', '메리어트 이그제큐티브 아파트먼트', '오리서원', '이화여자대학교', '양씨삼강문', 
           '은평구청', '동백해녀복지회관', '풍납토성', '단성사', '경북대학교 대구캠퍼스', '중랑구립정보도서관', '오라카이 송도 파크 호텔', '원효대사 깨달음 체험관', '덕풍5일장', '인천개항박물관', 
           '연수구청', '검단탑종합병원', '국립 한글 박물관', '롯데갤러리 본점', '동탄 복합문화센터', '온온사', '한국예술종합학교 석관동캠퍼스', '순천향대학교 부천병원', '현절사', '인천 구 대화조 사무소', 
           '대한민국 수준원점', '새마을 전통시장', '부산시청자미디어센터', '하남 덕풍전통 재래시장', '인창도서관', '가좌 도서관', '증권 박물관', '신성장로교회', '금천마을회관', '동묘벼룩시장', 
           '수영구 노인복지관', '부산광역시립해운대도서관', '증권박물관', '용동큰우물', '한준신도비', '현대백화점[본점]', '잠두봉유적', '서울금성당', '꿈벗도서관', '그랜드 하얏트 인천', 
           '노원정보도서관', '고양 독립운동 기념탑', '백인제가옥', '롯데백화점[영등포]', '아이파크몰', '부산환경공단남부사업소', '경교장', '용화사', '숭실대학교', '가회동성당', 
           '롯데마트 부평역점', '일산 밤가시초가', '정동교회', '안양 종합운동장', '송도오토캠핑장', '성신여자대학교', '연세대학교 신촌캠퍼스', '남구 청소년도서관', '인천개항장 근대건축전시관', '모래내시장', 
           '고양시 문예회관', '동덕아트갤러리', '석촌동백제초기적석총', '신매시장', '일산 라페스타', '효성도서관', '롯데백화점[건대스타시티]', '천호대교', '필리핀군 참전비', '올림픽대교', 
           '영등포구청', '연산8동 주민센터', '동탄 라마다 호텔', '노들섬', '원마운트', '코오롱 스페이스 K', '한중문화관', '성지파크 골프장', '파라다이스시티', '사당솔밭도서관', 
           '밀레니엄 힐튼 서울', '중앙대학교병원', '한양대학교 서울캠퍼스', '부산고등검찰청', '인천관동갤러리', '캠프 라과디아 체육공원', '남양주 시립 박물관', '소의문터', '의정부 의류타운', '연무관', 
           '좋은애인요양병원', '덕수궁 돌담길', '유석 조병옥선생 동상', '광주문화원', '사월역작은도서관', '곤지암 리조트', '손기정 기념관', '행복로 시민광장', '통일워터파크', '민락수변공원', 
           '강북구청', '돈의문터', '강남구청', '일산 차병원', '동래구청', '롯데호텔 서울', '송파도서관', '낙민공원', '성수대교', '화석정', 
           '동대문구청', '선사유적공원', '남양주시청 제1청사', '산본도서관', '명지대학교 인문캠퍼스', '정은재가복지센터', '남평화 상가', '부산대학교', '휴띠끄 차이나타운점', '인천 해돋이공원', 
           '이케아_광명', '국립현대미술관 고양레지던시', '개원사', '해양수산인재개발원', '과천 향교', '인천 LF스퀘어', '동대문역사관', '종각젊음의거리', '영화의전당', '쉐라톤 그랜드 인천 호텔', 
           '반석산 에코스쿨', '운현궁', '소악루', '복사골 문화센터 아트홀', '인천 검단선사박물관', '칼빈대학교', '하피랜드', '청라호수공원', '고양 어울림누리', '금암기적비', 
           '한류스타거리', '화원동산', '승동교회', '과천도시공사', '훼릭스 야외수영장', '양천향교', '삼송 도서관', '송암 박두성선생 생가 기념비', '강남세브란스병원', '용산구청', '성북구청', '원마운트 워터파크', '동작도서관', '이곡장미공원', '이대목동병원', '동구릉원릉', '고려대학교', '구러시아 공사관', '화랑대역 - 폐역', '달서구립 도원도서관', '현대백화점[무역센터]', '팽성읍 객사', '둔촌역 전통시장', '유니온파크 물놀이터', '일산 MBC 드림센터', '부평안전체험관', '융건릉', '이태원 앤틱 가구 거리', '유진민속 박물관', '인현시장', '남산도서관', '예송갤러리', '유식물원', '홈플러스 계산점', '베스트웨스턴 해운대 호텔', '구 인천일본제일은행지점', '신포 지하상가', '성산대교', '노보텔 앰배서더 서울 동대문 호텔', '한우물', '별마당 도서관', '고척스카이돔', '국립한국복지대학교', '동서울 승마클럽', '해운대중일교회', '양산도서관', '목동종합운동장', '부천 식물원', '양천도서관', '의정부 예술의전당', '금천구립가산도서관', '펭귄시장', '율목도서관', '동대문시장', '의정부 어린이 도서관', '한국항공대학교', '어린이 천문대', '중남미 문화원', '버텀라인', '보람공원', '오산역 환승터미널', '경희의료원', '주한중국문화원', '부경대학교수산과학연구소', '그랜드백화점 일산점', '제물포구락부', '양천구청', '오산 외삼미동고인돌', '고양시 종합운동장', '강남대학교', '수영구 생활문화센터', '잇다 스페이스', '63시티', '강북문화예술회관', '롯데백화점[캐슬플라자]', '이준열사동상', '부천시민회관 대문화관련', '서울시립대학교', '창경궁', '부천 여성청소년 재단', '베스트웨스턴 하버파크 호텔 인천', '보문사', '노원구청', '반여도서관', '정독도서관', '인천어린이과학관', '롯데백화점 일산점', '디아크', '남양시장', '북악팔각정', '아쿠아플라넷 (일산)', '송도국제기구도서관', '문화비축기지', '해방촌', '김포 종합운동장', '동안구청', '노량진 수산물 도매시장', '서대문구립이진아기념도서관', '대구오페라하우스', '송도해돋이도서관', '성균관대학교', '홈플러스 작전점', '동국대학교 (바이오메디 캠퍼스)', '부천체육관', '봉원사', '진고개', '동대문(흥인지문)', '광희 패션몰', '송도컨벤시아', '영안침례교회', '돈암시장', '북촌 한옥마을', '경기북부 지방경찰청', '메디플렉스 세종병원', '김포국제공항', '롯데백화점[잠실]', '김대중컨벤션센터역', '의정부 제일시장', '공예마을', '송파구청', '강서구청', '롯데백화점[달서구]', '부천 자유시장', '고양 어린이 박물관', '다니엘 종합병원', '인천대공원 반려동물 놀이터', '그랜드 하얏트 서울', '플레이 아쿠아리움 부천', '구리 신재생 에너지 홍보관', '고양 600 기념관', '대연중앙장로교회', '윤동주 문학관', '렛츠런팜 원당', '경안군 및 임창군묘', '안양 중초사지 당간지주', '구로시장', '궐리사', 'BIFC', '신세계 사이먼 시흥 프리미엄아울렛', '송산마을회관', '유엔조각공원', '서울원각사지십층석탑', '구산성지', '청혜원', '봉하마을회관', '홈플러스 송도점', '현대백화점[목동]', '부산시민공원역사관', '서울 웨스틴 조선 호텔', '신세계백화점[타임스퀘어]', '경기과천 교육도서관', '대화 도서관', '양화진 외국인 선교사 묘원', '인천백병원', '서영대학교 (파주캠퍼스)', '반여3동 행정복지센터', '국립부산국악원', '오이도 빨강등대', '인하대병원', '웅진플레이도시', '북악스카이웨이', '영등포중앙시장', '서울교육대학교', '청·일조계지 경계계단', '한국외국어대학교 서울캠퍼스', '트라이볼', 'JW 메리어트 호텔 서울', '부평향교', '내원암', '부평종합시장', '창의문 (자하문)', '송정동 행정복지센터', '롯데백화점[청량리]', '하남문화원', '선바위 미술관', '스퀘어원', '명장한마음공원', '서울풍물시장', '양재도서관', '하남시 청소년 수련관', '생명의나무 전망대', '인천광역시 여성문화회관', '영천시장', '경기대학교 서울캠퍼스', '사직단', '성동구청', '한국생산기술연구원 해양로봇센터', '반고개 무침회 골목', '광희문', '꿈빛 도서관', '킨텍스', '건원릉', '동부성서침례교회', '조동호집터', '뉴코아아울렛 부천점', '인천상륙작전 표지석 그린비치', '광통교', '광화문 광장', '호가정', '송도커넬워크', '강북문화정보도서관', '망원 한강공원', '롯데시네마[상인]', '조형갤러리', '한림병원', '맥도생태공원', '백영수 미술관', '고양시청', '약천사', '명지병원', '아인스월드', '인제대학교 일산 백병원', '구리 시립 토평도서관', '부산지방검찰청 서부지청', '의정부 종합운동장', '망원정지', '서울 월드컵 경기장', '씨제이 이엔엠센터', '온누리종합병원', '한국근대문학관', '부천 유럽 자기 박물관', '봉은사', '봉화산', '마천중앙시장', '석촌동 고분군', '신흥시장', '꽃님어린이공원', '지장암', '부산금사요양병원', '부평역사쇼핑몰', '인천 차이나타운', '해운대교육지원청', '부천 한국만화영상 진흥원', '계양구청', '돈의문 박물관 마을', '학고재', '도봉구청', '블루스퀘어', '파크하얏트 서울', '가야대교', '학림사', '이화 벽화마을', '영동전통시장', '양동시장', '문현2동 주민센터', '통일 워터파크', '스미스평화관', '인천 답동성당', '홍은도담도서관', '원당시장', '스타힐 리조트', '서울 신라호텔', '종로신진시장', 'IT도서관', '부평구청', '한국 카메라 박물관', '인천일본제18은행지점', '남구장애인복지관', '문학박태환수영장', '서울숲', '반여3동전통시장', '화성시 미디어센터', '부산시민공원', '인천학생6.25참전관', '대구 삼성 라이온즈 파크', '강남차병원', '마포대교', '한우리교회', '연꽃테마파크', '남구로시장', '삼일독립선언유적지', '광주 남구 청소년 수련관(문화정보 도서관, 문화원, 남구 생활문화센터 포함)', '시흥 문원리 삼층석탑', '소래역사관', '하남역사박물관', '북성포구', '혜명단청박물관', '평택시 문화재단', '더 플라자', '삼청공원 숲속도서관', '갤러리수정', '부평아트센터', '인터컨티넨탈 서울 코엑스', '부산광역시립명장도서관', '삼어행복마을센터', '모라공원', '용호성당', '마루광장', '모라어울림공원', '고동바위공원', '인천광역시교육청북구도서관', '현대백화점[신촌]', '고양 가와지 볍씨 박물관', '부천 천문 과학관', '연산군 시대금표비', '부천 시립 북부 도서관', '소다 미술관', '경인교육대학교 (경기캠퍼스)', '서울한양도성', '청운문학도서관', '현대백화점 킨텍스점', '상록구청', '원효대교', '삼청공원', '롯데백화점[미아]', '중랑동부시장', '양주 시립 장욱진 미술관', '숭례문', '진흥종합시장', '인하대학교 정석학술정보관', '김중업 건축박물관', '증산힐스', '창덕궁', '퓨전문화 관광홍보관', '롯데아울렛 이시아폴리스', '화성시 어린이문화센터', '더 클래식 500', '배다리 안내소', '정동1928 아트센터', '금정문화회관', '인사아트프라자갤러리', '한강유역환경청', '서대문구청', '농협대학교', '롯데월드', '경의선책거리', '세종대학교', '청담삼익시장', '무양서원', '한전아트센터 갤러리', '스위스 그랜드 호텔', '해상문화공연장', '홀리데이 인 인천 송도', '시흥대교', '원인재', '고려대학교구로병원', '부천전수관', '해동성취사', '제물진두 순교기념경당', '송도 센트럴파크', '한국 등잔 박물관', '전태일 기념관', '한양도성박물관', '강매동석교', '선인장 다육식물 연구소', '광주광역시 서구청', '부산한마음스포츠센터', '롯데백화점 중동점', '부천 고려 호텔', '홍예문', '하남 문화예술회관', '홍법사', '갤러리 현대 본관', '아트파라디소 파라다이스시티', '문래창작촌', '평택대학교', '의정부시청', '안락누리도서관', '동대문 디자인 플라자', '중부대학교 (고양캠퍼스)', '금나래아트홀도서관', '한국원자력의학원원자력병원', '흥인지문공원', '가회민화박물관', '롯데백화점[노원]', '꿈꾸는어린이도서관', '소래포구', '흥국사', '구 일본우선(郵船)주식회사 인천지점', '쌈지길', '영종도서관', '부산상공회의소', '장충단비', '곰달래도서관', '천주교안락교회', '암사동선사주거지', '약현성당', '송현관광시장', '죽동1구마을회관', '소전서림', '웃는얼굴아트센터', '미추홀구청', '공세리 오층석탑', '초록공원', '서울과학기술대학교', '광진정보도서관', '지구촌 민속 박물관', '부천 승마장', '덕수궁', '스포츠 몬스터 고양', '동구릉혜릉', '도산 안창호 기념관', '천주교이기대성당', '우정총국', '투나쇼핑몰', '연안부두 전망대', '선학별빛도서관', '수성구립 고산도서관', '한국체육대학교', '메이필드 호텔 앤 리조트', '한가람미술관', '현대시장', '서강대교', '안양박물관', 'G 챔피언스 파크', '시그니엘 서울', '애관극장', '남한산성', '광주학생독립운동기념공원(광주학생독립운동역사관, 광주학생독립운동기념탑 포함)', '사문진나루터', '과천 시민회관', '한국이민사박물관', '마포구청', '낙산공원', '반포대교', '망우리공원', '의정부 실내빙상장', '도당어울마당', '하늘공원', '경운궁중명전', '하남시청', 'IKEA동부산점', '호림 박물관', '팔당역', '동덕여자대학교', '유한대학교', '문화공원', '건국대학교병원', '낙산 묘각사', '시흥시청', '연수체육공원', '미추홀공원', '상상아트센터', '서울 역사 박물관', '여래사', '김대중컨벤션센터', '늘푸른교회', '한성 백제 박물관', '한양공원표석', '길상사', '부산성모병원', '부평깡시장', '조계사', '대각사', '주렁주렁 하남점', '이승엽 야구장', '교하향교', '서초구청', '롯데마트 계양점', '행주서원', '프리지언 클럽앤리조트', '행주 서원', '서울중앙시장', '덕양서원', '서울대학교병원', '안락꽃밭공원', '호텔 아쿠아펠리스', '고양 밥할머니 석상', '용주 서원', '인천대공원 목재문화체험장', '기준선생묘', '한남대교', '일산 스프링힐스 CC', '노보텔 스위트 앰배서더 서울 용산', '강남도서관', '모아뮤지엄', '소노캄 고양', '우림시장', '창신동 절개지', '경리단길', '롯데백화점[부산본점]', '부평역사박물관', '원더박스', '도선사', '환구단', '민락회타운', '장미란 체육관', '남포동건어물도매시장', '미추홀도서관', '짜장면박물관', '국립중앙도서관', '안산시청', '초연다구박물관', '올림픽공원 세계평화의문', '삼전도비', '구리타워', '노작 홍사용 문학관', '연산성서침례교회', '메세나폴리스몰', '카톨릭대학교여의도성모병원', '국민대학교', '고려 공양왕릉', '연지동 주민센터', '한강대교', '세종대왕 동상', '장경사', '경기도 박물관', '파주시청', '후암시장', '삼천사', '스페이스크리오', '만안구청', '평화공원', '경릉', '스트리트뮤지엄', '롯데월드타워', '영휘원과 숭인원', '서경대학교', '독립문', '풍납리토성', '평화시장', '소래습지생태공원', '부천 종합터미널', '롯데마트 삼산점', '광장시장', '대구실내체육관', '하남 종합운동장', '화계사', '고양향교', '신세계백화점[본점]', '서울 남현동 요지', '한국 만화박물관', '대한의원', '서삼릉', '남구국민체육센터', '양화대교', '유애서원', '남대문시장', '송도국제어린이도서관', '구공업전습소본관', '이태원 부군당', '경기북부 병무지청', '아세안문화원', '포시즌스 호텔 서울', 'NC백화점[송파]', '나사렛병원', '덕양구청', '고양문화원', '동국대학교 서울캠퍼스', 'IFC몰', '동작구청', '올림픽공원', '이천 세라피아', '신장 전통시장', '한누리교회', '안양 중초사지 삼층석탑', '광주 삼리 구석기유적', '작전도서관', 'KBS 온', '디큐브아트센터', '역대현감비석군', 'NC백화점[강서]', '롯데백화점[관악]', '문화역 서울 284', '수영구청', '겸재정선미술관', '파라다이스 씨티 씨메르(인천)', '추억극장미림', '운남동 근린공원', '화교역사관', '국립어린이청소년도서관', '국립 4.19 민주 묘지', '부천역 지하도상가', '인천내동성공회성당', '부천 종합운동장', '서울장신대학교', '부천 상동시장', '평택시립 배다리 도서관', '구리 남양주 교육지원청', '선릉', '소울음 아트센터', '현대백화점[천호]', '오크우드 프리미어 인천 호텔', '레솔레파크', '관악구청', '광산구 쌍암공원', '4.19학생혁명기념탑', '선잠단지', '인제대학교 해운대백병원', '광주광역시 남구청', '어린이 건강식생활 체험관', '역말 문화회관', '인제대학교상계백병원', '인천상륙작전 표지석 레드비치', '장충체육관', '부천 도시공사', '인천대공원', '잠실종합운동장', '현대 모터 스튜디오 고양', '북구청', '천주교장산성당', '동작대교', '의정부 시외버스 터미널', '일산시장', '군포 시외버스 터미널', '안양 문화원', '배다리공예상가', '신호마을회관', '인천시역사자료관', '배다골 테마파크', '국립중앙박물관', '암사종합시장', '평택시청', '안양 석수동 마애종', '풍동 도서관', '김포대학교', '부산광역시청', '홈플러스 인하점', '뉴코아아울렛[강남]', '만해기념관', '광주시 청소년 수련관', '서울특별시청', '국민건강보험공단 부산남부지사', '청담대교', '만기사', '굴포천 생태하천', '인천항크루즈터미널', '인천도시역사관', '대천공원', '중부시장', '강서구립가양도서관', '이화벽화마을', '송도 센트럴 파크 호텔', '한마음교회', '명릉', '정릉', '세빛섬', '원당성당', '중구청', '서면삼정타워', '덕성여자대학교', '광운대학교', '부천 옹기 박물관', '부천 아트벙커 B39', '한전아트센터', '의정부 미술도서관', '기술보증기금', '윤봉길 의사 기념관', '신세계백화점[강남]', '르 메르디앙 서울', '건국대학교 서울캠퍼스', '광화문', '순천향대학교서울병원', '강서구립등빛도서관', '관룡탑', '국회 의사당', '다이노스타', '용호남장로교회', '이곤선생 효자문', 'N 서울타워', '카톨릭대학교은평성모병원', '백남준 아트센터', '이화마을', '월미테마파크', '남가좌새롬어린이도서관', '경희궁', '포마 자동차 디자인 미술관', '국립 과천 과학관', '국민건강보험공단 일산병원', '안심공원', '계양문화회관', '홀리데이인 광주호텔', '양산호수공원', '종로구청', 'NC백화점[구로]', '혜화문 (홍화문)', '서울도서관', '하남시 미사도서관', '현대백화점[미아]', '소래철교', '오산 시립 햇살마루도서관', '연산4동행정복지센터', '오산 시립미술관', '현대백화점 중동점', '잠수교', '보광사', '서울아산병원', '희망 Library Center', '전차381호', '현대프리미엄아울렛', '내리교회', '역사문화공원', '일산 벨라시타', '정약용 유적지', '고양시립 화정도서관', '행주대첩비', '필리핀군 참전 기념비', '대한서림', '최규하대통령가옥', '종로타워', '구리 아트홀', '루터대학교', '좋은강안병원', '경희대학교 (국제캠퍼스)', '어을미센터', '재송어린이도서관', '길음시장', '부수지 공원', '베스트웨스턴 프리미어 인천 에어포트 호텔', '원광대학교의과대학산본병원신관', '인천성모병원', '경복궁', '반야월 연꽃단지', '부산예술회관', '취병조형유허비', '국청사', '부개도서관', '보신각', '구서울대학교본관', '아이즈빌아울렛', '안양시청', '주엽 어린이 도서관', '신한대학교 (의정부캠퍼스)', '강북삼성병원', '롯데백화점[강남]', '성동구립도서관', '(구)인천일본제58은행지점', '시립 백석도서관', '안락2동행정복지센터', '충무공 이순신 동상', '동주경로당', '남산골 한옥마을', '동국대학교 일산병원', '세흥시장', '고양 아람누리', '오산시 유엔군 초전기념관', '남동구청', '봉산문화거리', '마장 축산물시장', '기쁨의샘교회', '현대백화점 유플렉스 중동점', '부산지방우정청', '해운대한빛요양병원', '상록도서관', '달서가족문화도서관', '광개토태왕동상', '괴정2동행정복지센터', 'KT&G 상상마당', '웃음누리공원', '부산지방법원[서부지원]', '잠심대교', '수유전통시장', '포돌이 안전도서관', '중랑구청', '동묘', 'APEC나루공원', '몽촌토성', '백범김구선생상', '월미 관광특구', '인천중동우체국', '용비교', '미호', 'G타워', '유엔군 초전기념비', 'NC백화점[불광]', '동호대교', 'JW 메리어트 동대문 스퀘어 서울', '천주성삼병원', '상무조각공원', '신트리공원', '신세계백화점 경기점', '롯데마트 송도점', '만안교', '가락시장', '인왕시장', '대동법 시행비', '사직야구장', '부산은행', '북악산 서울 성곽', '양주시청', '시흥 오이도 선사 유적 공원', '허밍웨이길', '고양아람누리 아람미술관', '송상현광장', '부천 중동시장', '동구릉휘릉', '인천기독병원', '화원장터', '남양주시청 제2청사', '롯데백화점[김포공항]', '성녕대군묘 및 신도비', '삼봉기념관', '구리시청', '가든파이브', '펄벅 기념관', '이야기꽃도서관', '성결대학교', '부산전자종합시장', '꿈두레 도서관', '종묘', '카톨릭대학교서울성모병원', '인천학생교육문화회관', '남양리 신빈 김씨 묘역', '이화여자대학교의료원', '마리나 베이 서울', '안양대학교', '뚝섬 전망문화 콤플렉스 J-Bug', '강동구청', '코엑스', '부천시청', '프라움 악기 박물관', '한성대학교', '반여4동 주민센터', '온천엘리바덴', '이케아_고양', '행주대첩 기념관', '인천아트플랫폼', '정부고양지방합동청사', '롯데백화점[본점]']


In [ ]:
EPOCHS = 20
BATCH_SIZE = 32
SKIP_VALIDATION = False  ## Train, Validation 데이터 모두 학습 데이터로 사용하고 싶을 경우 True, Validation 사용할 경우 False
IMAGE_SIZE = [540, 960]
RESIZE_SIZE = [600, 600]

INIT_LR = 5e-5
MOMENTUM = 0.9
DECAY = 1e-5

if SKIP_VALIDATION:
    TRAINING_FILENAMES = TRAINING_FILENAMES + VALIDATION_FILENAMES

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.chdir('/content/drive/MyDrive')

In [ ]:
# Google Cloud에서 학습을 위해 데이터를 읽오오고 데이터셋을 만든다.

def decode_image(image_data, ):
    image = tf.image.decode_image(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        'image/height': tf.io.FixedLenFeature([], tf.int64, default_value=0),
        'image/width': tf.io.FixedLenFeature([], tf.int64, default_value=0),
        'image/colorspace': tf.io.FixedLenFeature([], tf.string, default_value=''),
        'image/channels': tf.io.FixedLenFeature([], tf.int64, default_value=0),
        'image/format': tf.io.FixedLenFeature([], tf.string, default_value=''),
        'image/id': tf.io.FixedLenFeature([], tf.string, default_value=''),
        'image/encoded': tf.io.FixedLenFeature([], tf.string, default_value=''),
        'image/class/label': tf.io.FixedLenFeature([], tf.int64, default_value=0)
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image/encoded'])
    label = tf.cast(example['image/class/label'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        'image/height': tf.io.FixedLenFeature([], tf.int64, default_value=0),
        'image/width': tf.io.FixedLenFeature([], tf.int64, default_value=0),
        'image/colorspace': tf.io.FixedLenFeature([], tf.string, default_value=''),
        'image/channels': tf.io.FixedLenFeature([], tf.int64, default_value=0),
        'image/format': tf.io.FixedLenFeature([], tf.string, default_value=''),
        'image/id': tf.io.FixedLenFeature([], tf.string, default_value=''),
        'image/encoded': tf.io.FixedLenFeature([], tf.string, default_value='')
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image/encoded'])
    idnum = example['image/id']
    return image, idnum # returns a dataset of image(s)

def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

def data_augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.resize(image, RESIZE_SIZE)

    return image, label   
  
def data_resize(image, label):
    image = tf.image.resize(image, RESIZE_SIZE)
    # 입력 이미지를 모델에 맞게 resize
    
    return image, label

def get_training_dataset(do_aug=True):
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
    dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    # 다음 batch에 사용할 데이터를 학습 중 미리 prefetch
    dataset = dataset.prefetch(AUTO)
    return dataset

def get_validation_dataset(ordered=False):
    dataset = load_dataset(VALIDATION_FILENAMES, labeled=True, ordered=ordered)
    dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    # 다음 batch에 사용할 데이터를 학습 중 미리 prefetch
    dataset = dataset.prefetch(AUTO)
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.map(data_resize)
    dataset = dataset.batch(BATCH_SIZE)
    # 다음 batch에 사용할 데이터를 학습 중 미리 prefetch
    dataset = dataset.prefetch(AUTO)
    return dataset


# NUM_TRAINING_IMAGES = 88102 # count_data_items(TRAINING_FILENAMES)
    # 벨리 포함
NUM_TRAINING_IMAGES = 88102-17621 # count_data_items(TRAINING_FILENAMES)
NUM_VALIDATION_IMAGES = 17621 #count_data_items(VALIDATION_FILENAMES)
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
VALIDATION_STEP = NUM_VALIDATION_IMAGES // BATCH_SIZE
print('Dataset: {} training images, {} validation images'.format(NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES))

# Model

In [ ]:
from tensorflow.keras.mixed_precision import experimental as mixed_precision

policy = mixed_precision.Policy('mixed_bfloat16')
mixed_precision.set_policy(policy)

print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

In [ ]:
# 이전에 학습된 가중치 로드하는 부분

import datetime

MODEL_NAME = 'sampleCNN'
MODEL_PATH = 'model/'
checkpoint_path = os.path.join(MODEL_PATH, MODEL_NAME)
os.makedirs(checkpoint_path, exist_ok=True)
model_file_path = os.path.join(checkpoint_path, 'Epoch_{epoch:03d}_Val_{val_loss:.3f}.hdf5')
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=model_file_path, monitor='val_loss', verbose=1, save_best_only=True)

checkpoint_path = os.path.join(MODEL_PATH, MODEL_NAME)
weigth_file = tf.io.gfile.glob('{}/*.hdf5'.format(checkpoint_path))[-1]
print(weigth_file)


In [ ]:
# 학습에 이용할 모델을 설정한다.
with strategy.scope():

    enet = tf.keras.applications.EfficientNetB6(
        input_shape=[*RESIZE_SIZE, 3],
        weights='imagenet' ,
        include_top=False
    )
    enet.trainable = True

    model = tf.keras.Sequential([
        enet,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(len(CLASSES)),
        tf.keras.layers.Activation('softmax', dtype='float32')
    ])

    if checkpoint_path:
        model.load_weights(weigth_file)
        print('-------------Load model - {}-------------'.format(weigth_file))
    else:
        print('-------------From Scratch mode-------------')

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=INIT_LR), #, momentum=MOMENTUM, decay=DECAY),
        loss = 'sparse_categorical_crossentropy',
        metrics=['sparse_categorical_accuracy']
    )
    

In [ ]:
model.summary()

In [ ]:
# import datetime

# checkpoint_filepath = os.path.join(OUTPUT_PATH, 'weights', 'checkpoint_{epoch}')
# os.makedirs(checkpoint_filepath, exist_ok=True)
# model_file_path = os.path.join(checkpoint_filepath, 'Epoch_{epoch:03d}_Val_{val_loss:.3f}.hdf5')
# model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_filepath,
#     monitor='val_loss',
#     save_best_only=True
#     )

# log_dir = os.path.join(OUTPUT_PATH, 'logs')
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
history = model.fit(
    get_training_dataset(),
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=[checkpoint],
    validation_data=get_validation_dataset(),
    validation_steps=VALIDATION_STEP
    
)

In [ ]:
# # 훈련 결과를 확인합니다.
plt.plot(history.epoch, history.history['loss'], '-o', label='training_loss')
plt.plot(history.epoch, history.history['val_loss'], '-o', label='validation_loss')
plt.legend()
plt.xlim(left=0)
plt.xlabel('epochs')
plt.ylabel('loss')
plt.show()

In [ ]:
plt.plot(history.epoch, history.history['sparse_categorical_accuracy'], '-o', label='training_accuracy')
plt.plot(history.epoch, history.history['val_sparse_categorical_accuracy'], '-o', label='validation_accuracy')
plt.legend()
plt.xlim(left=0)
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()

In [ ]:
# 가장 좋은 모델의 weight를 불러옵니다.
checkpoint_path = os.path.join(MODEL_PATH, MODEL_NAME)
weigth_file = tf.io.gfile.glob('{}/*.hdf5'.format(checkpoint_path))[-1]
print(weigth_file)
model.load_weights(weigth_file)

# 예측 수행
test_dataset = tf.data.TFRecordDataset(TEST_FILENAMES)
test_dataset = test_dataset.map(read_labeled_tfrecord).batch(64).prefetch(1)
y_pred = model.predict(test_dataset)

In [ ]:
import pandas as pd
import os.path as pth

MODEL_NAME = 'result'
# 예측 결과로 제출 파일을 생성합니다.
submission = pd.read_csv('data/submission.csv', index_col=0)

pred_labels = np.argmax(y_pred, axis=1) 
pred_probs = np.array([pred[indice] for pred, indice in zip(y_pred, pred_labels)])

submission['landmark_id'] = pred_labels
submission['conf'] = pred_probs

today_str = datetime.date.today().strftime('%Y%m%d')
result_filename = '{}.csv'.format(MODEL_NAME)
submission_csv_fileaname = pth.join('data/', '_'.join([today_str, result_filename]))
submission.to_csv(submission_csv_fileaname, index=True)